In [11]:
import spotipy
import chardet
from typing import Dict,List
import pandas as pd

In [24]:
cid = 'ca54418c3218402b95563af47a745061'
secret = '1fd59395ee214f77ab52c05864bd2860'
redirect_uri = 'http://localhost:8080/callback'
ccm = spotipy.SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=ccm)

In [25]:
#제목으로 audio feature 쿼리 요청
def get_audio_features(song_title:str)->Dict:
    query = f"track:{song_title}"
    result = sp.search(q=query, type="track", limit=1)
    
    if result['tracks']['items']:
        track = result['tracks']['items'][0]
        track_id = track["id"]  
        audio_features = sp.audio_features(track_id)[0] 
        return audio_features

#제목으로 artist genre 쿼리 요청
def get_artist_genres(title:str)->Dict:
    result = sp.search(q=f"track:{title}", type="artist", limit=1)
    if result["artists"]["items"]:
        artist = result["artists"]["items"][0]  
        artist_name = artist["name"]
        genres = artist["genres"]  
        return {"artist": artist_name, "genres": genres}

In [ ]:
path = "./Data/preparation.csv"
with open(path,"rb") as f:
    encoding = chardet.detect(f.read())["encoding"]

df = pd.read_csv(path,encoding=encoding)

In [27]:
for index, row in df.iterrows():
    if not pd.isna(row["danceability"]):
        continue
    features = get_audio_features(row["Song"])
    if not features:
        continue     
    for feature, value in features.items():
        df.at[index, feature] = value

In [54]:
for index, row in df.iterrows():
    if not pd.isna(row["artist genres"]):
        continue
    features = get_artist_genres(row["titles"])
    if not features:
        continue
   
    for feature, value in features.items():
        df.at[index, feature] = value

In [56]:
df[["artist genres","danceability"]].head()

,artist genres,danceability
0,"['adult standards', 'easy listening']",0.247
1,['british dance band'],0.344
2,"['vocal harmony group', 'lounge']",0.531
3,"['adult standards', 'lounge', 'easy listening']",0.670
4,"['space age pop', 'honky-tonk piano']",0.332


In [58]:
df.to_csv("./Data/prepared.csv")